In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
# load model file and history of the model
saved_dir = 'D:/JATHURSH/Jupyter_notebook_projects/DrowsinessDetectionOnCombinedDataset/saved_data/models/'

model = tf.keras.models.load_model(saved_dir + 'dr_det_model_1.h5')

with open(saved_dir + 'dr_det_model_1_history.pkl', 'rb') as file:
    history = pickle.load(file)

In [3]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow import lite

# Convert the Keras model to TensorFlow Lite with float16 precision
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.target_spec.supported_types = [tf.float16]
converter._experimental_lower_tensor_list_ops = False
tflite_fp16_model = converter.convert()

# Load the converted TensorFlow Lite model with float16 precision
interpreter = tf.lite.Interpreter(model_content=tflite_fp16_model)
interpreter.allocate_tensors()

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmp92oj5y6g\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmp92oj5y6g\assets


In [4]:
# Load the pre-trained Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the pre-trained Haar Cascade classifier for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [5]:
from pygame import mixer

# Initialize the mixer module
mixer.init()

alarm_sound_path = 'D:/JATHURSH/Jupyter_notebook_projects/DistractionDetectFinal/saved_data/test_assets/alarm.mp3'

# Load the alarm sound
mixer.music.load(alarm_sound_path)

# # Play the alarm sound
# mixer.music.play()

# # Wait for a few seconds (you can adjust the duration)
# time.sleep(2)

# # Stop the alarm sound
# mixer.music.stop()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [11]:
import time

# Initialize the video capture from the default camera (0) or a video file
cap = cv2.VideoCapture(0)  # Use 0 for default camera, or provide the video file path

current_state_start_time = None
warning_duration_threshold = 2
    
while True:
    # Capture video frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        break  # Break the loop if no frame is captured
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Crop the face region for eye detection
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]
        
        # Detect eyes in the face region
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=4)
        
        for (ex, ey, ew, eh) in eyes:
            # Draw a rectangle around the detected eye
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (255, 0, 0), 2)
            
            # Crop the eye region
            eye_roi = roi_gray[ey:ey + eh, ex:ex + ew]
            
            # Convert the grayscale image to RGB format
            rgb_eye_roi = cv2.cvtColor(eye_roi, cv2.COLOR_GRAY2RGB)
            
            # Resize the eye image to 224x224 pixels
            eye_resized = cv2.resize(rgb_eye_roi, (224, 224))
            
            # Adding the 4th dimension for compatibility with the mobileNetV2 model
            final_eye_image = np.expand_dims(eye_resized, axis=0)
            
            # Normalize - using the same mobileNetV2's preprocess_input method
            final_eye_image_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(final_eye_image)
            
            # Set the input tensor for the model
            interpreter.set_tensor(interpreter.get_input_details()[0]['index'], final_eye_image_preprocessed)

            # Run inference
            interpreter.invoke()
            
            drowsiness_prediction = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])
            
            # Check if the prediction indicates drowsiness
            is_awake = (drowsiness_prediction >= 0.5).astype(int)
            
            # Display the prediction result on the frame
            label = "Awake" if is_awake else "Drowsy"
            color = (0, 255, 0) if is_awake else (0, 0, 255)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            drowsiness_duration = 0
            alarm_triggered = False
            frame_rate = 30

            if not is_awake:
                elapsed_time = time.time() - current_state_start_time
                if elapsed_time >= warning_duration_threshold:
                    if not mixer.music.get_busy():
                        warning_message = "Warning: Driver is drowsy!"
                        cv2.putText(frame, warning_message, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        mixer.music.play()
            else:
                current_state_start_time = time.time()
                mixer.music.stop()
                
    # Display the frame with detections and predictions
    cv2.imshow("Drowsiness Detection", frame)
    
    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()